# Code Interpreter : Exécution de Code avec OpenAI

Le **Code Interpreter** est un outil intégré qui permet au modèle d'exécuter du code Python dans un environnement sandbox sécurisé. Il est particulièrement utile pour l'analyse de données, les calculs complexes, et la génération de visualisations.

**Objectifs :**
- Activer et utiliser le code_interpreter
- Analyser des données automatiquement
- Générer des graphiques et visualisations
- Manipuler des fichiers uploadés

**Prérequis :** Notebook 1 (OpenAI Intro)

**Durée estimée :** 55 minutes

In [ ]:
%pip install openai python-dotenv pandas matplotlib -q

import os
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd

load_dotenv('../.env')
client = OpenAI()

# Charger le modèle depuis .env ou utiliser gpt-5-mini par défaut
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

print("Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")
print(f"Mode batch: {BATCH_MODE}")

### Interprétation de l'initialisation

L'initialisation a chargé les composants suivants :

| Composant | Rôle |
|-----------|------|
| `openai` | Client API pour interagir avec OpenAI |
| `python-dotenv` | Chargement sécurisé des clés API depuis `.env` |
| `pandas` | Manipulation de données tabulaires |
| `matplotlib` | Génération de visualisations (graphiques) |

**Variables d'environnement importantes** :
- `OPENAI_API_KEY` : Authentification API (obligatoire)
- `OPENAI_MODEL` : Modèle par défaut (ex: gpt-5-mini)
- `BATCH_MODE` : Mode non-interactif pour tests automatisés

Le message "Client OpenAI initialisé !" confirme que la connexion est établie et prête pour utiliser le Code Interpreter.

## Configuration de l'environnement

Dans cette section, nous initialisons le client OpenAI avec les configurations nécessaires.

**Variables d'environnement importantes** :
- `OPENAI_API_KEY` : Clé d'API OpenAI (obligatoire)
- `OPENAI_MODEL` : Modèle par défaut (recommandé: gpt-5-mini)
- `BATCH_MODE` : Mode batch pour exécution automatisée sans interaction

**Note** : Le Code Interpreter nécessite un modèle supportant les outils avancés (gpt-4, gpt-5-mini, gpt-4-turbo).

## Architecture du Code Interpreter

Le Code Interpreter fonctionne dans un **environnement sandbox Python sécurisé** qui offre :

### Bibliothèques disponibles
- **Analyse de données** : pandas, numpy, scipy
- **Visualisation** : matplotlib, seaborn, plotly
- **Machine Learning** : scikit-learn
- **Traitement d'images** : PIL, opencv
- **Mathématiques** : sympy, statsmodels

### Limitations importantes
- ❌ **Pas d'accès réseau** : impossible de télécharger des données externes
- ⏱️ **Timeout** : exécutions limitées dans le temps (environ 120 secondes)
- 💾 **Stockage temporaire** : les fichiers générés sont éphémères
- 🔒 **Sandbox isolé** : pas d'accès au système de fichiers externe

### Cas d'usage typiques
1. Analyse exploratoire de données (EDA)
2. Création de graphiques et visualisations
3. Calculs mathématiques complexes
4. Transformation et nettoyage de données
5. Génération de rapports automatisés

## Activation du Code Interpreter

Le Code Interpreter est disponible **uniquement via l'Assistants API** (pas via Chat Completions).

**Workflow typique :**
1. Créer un Assistant avec `tools=[{"type": "code_interpreter"}]`
2. Uploader les fichiers nécessaires via Files API
3. Créer un Thread et envoyer des messages
4. Exécuter le Run et récupérer les résultats

**Note importante :** Contrairement aux functions/tools classiques de Chat Completions, le code_interpreter n'est pas directement accessible via `client.chat.completions.create()`. C'est une fonctionnalité exclusive de l'Assistants API.

## Pourquoi utiliser l'Assistants API ?

Vous vous demandez peut-être : **pourquoi cette complexité avec l'Assistants API au lieu de simplement utiliser Chat Completions ?**

**Réponse** : Le Code Interpreter est un outil **stateful** qui nécessite :
- Un **environnement persistant** pour exécuter du code
- Une **gestion de fichiers** uploadés et générés
- Un **contexte partagé** entre plusieurs exécutions
- Des **capacités sandbox** sécurisées

L'Assistants API fournit cette infrastructure. Chat Completions est **stateless** (sans mémoire entre appels), donc inapproprié pour ce cas d'usage.

**Analogie** : 
- Chat Completions = conversation téléphonique (pas de mémoire)
- Assistants API = session de travail collaborative (mémoire + ressources partagées)

### Interprétation de la démonstration

Cette démonstration illustre deux points importants :

**1. Limitation de Chat Completions** :
- Le type `'code_interpreter'` n'est PAS accepté dans `tools=[]`
- Seuls `'function'` et `'custom'` sont supportés
- Erreur typique : `Invalid tool type: 'code_interpreter'`

**2. Alternative via génération de code** :
- Le modèle peut **générer** du code Python (comme démontré)
- Mais il ne peut PAS **exécuter** ce code
- C'est une limitation fondamentale de Chat Completions

**Exemple de sortie** : Le modèle génère le code pour calculer les nombres de Fibonacci, mais ne l'exécute pas. Vous devriez voir du code Python brut comme output, pas les résultats numériques.

**Pourquoi cette architecture ?** :
- **Chat Completions** = stateless, rapide, conversationnel
- **Assistants API** = stateful, avec ressources (fichiers, environnement d'exécution)

Pour exécuter réellement du code, passons à l'Assistants API dans les cellules suivantes.

In [ ]:
# Note: Le Code Interpreter n'est pas disponible via Chat Completions
# Cette cellule démontre ce qui NE fonctionne PAS

print("=== Note importante sur Code Interpreter ===")
print()
print("Le code_interpreter n'est PAS disponible via Chat Completions:")
print("  tools=[{'type': 'code_interpreter'}]  # ERREUR avec chat.completions.create")
print()
print("Seuls ces types d'outils sont supportés par Chat Completions:")
print("  - 'function' : Appel de fonctions personnalisées")
print("  - 'custom' : Outils personnalisés")
print()
print("Pour utiliser code_interpreter, utilisez l'Assistants API (voir cellules suivantes).")
print()

# Démonstration de ce qui fonctionne - calcul simple via le modèle lui-même
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "user", "content": "Calcule les 20 premiers nombres de Fibonacci et affiche-les sous forme de liste."}
    ],
    max_tokens=300
)

print("=== Alternative: Génération de code par le modèle (sans exécution) ===")
print(response.choices[0].message.content)

### Préparation du dataset de test

Avant de tester le Code Interpreter, créons un dataset réaliste de ventes sur 100 jours.

**Caractéristiques du dataset** :
- 100 jours de ventes (Jan-Avril 2025)
- Ventes aléatoires entre 100-500 avec une composante sinusoïdale (saisonnalité simulée)
- 4 régions géographiques
- 3 types de produits

Ce dataset servira de base pour démontrer les capacités d'analyse du Code Interpreter.

## Analyse de données automatisée

Le Code Interpreter excelle dans l'analyse de datasets. Il peut :
- Charger et explorer des fichiers CSV, Excel, JSON
- Calculer des statistiques descriptives
- Détecter des patterns et anomalies
- Générer des rapports structurés

Commençons par créer un dataset de test représentant des ventes sur 100 jours.

In [ ]:
# Créer un dataset de ventes
import pandas as pd
import numpy as np

np.random.seed(42)
dates = pd.date_range('2025-01-01', periods=100, freq='D')
data = {
    'date': dates,
    'ventes': np.random.randint(100, 500, 100) + np.sin(np.arange(100) * 0.1) * 50,
    'region': np.random.choice(['Nord', 'Sud', 'Est', 'Ouest'], 100),
    'produit': np.random.choice(['A', 'B', 'C'], 100)
}
df = pd.DataFrame(data)
df.to_csv('ventes_test.csv', index=False)

print(f"Dataset créé: {len(df)} lignes")
print("\nAperçu des données:")
print(df.head(10))
print("\nStatistiques de base:")
print(df.describe())

### Interprétation de la création du dataset

Le dataset a été créé avec succès. Analysons sa structure :

**Dimensions** :
- 100 lignes (observations journalières)
- 4 colonnes : date, ventes, region, produit

**Caractéristiques des données** :
- **Ventes** : Valeurs entre ~70 et ~525 (moyenne ~321)
- **Composante aléatoire** : `np.random.randint(100, 500, 100)`
- **Composante sinusoïdale** : `np.sin(...) * 50` simule une saisonnalité
- **Écart-type** : ~120 indique une variabilité importante

**Distribution géographique** :
- 4 régions équiprobables (Nord, Sud, Est, Ouest)
- 3 types de produits (A, B, C)

**Pourquoi cette structure ?** :
- **Saisonnalité** : Le terme sinusoïdal simule des cycles de vente réalistes
- **Randomisation** : Reproduit la variabilité naturelle des ventes
- **Seed fixe** (`np.random.seed(42)`) : Assure la reproductibilité des résultats

Ce dataset servira de base pour démontrer les capacités d'analyse du Code Interpreter dans les cellules suivantes.

### Interprétation des résultats d'analyse

L'assistant a automatiquement :

1. **Chargé le CSV** : Reconnaissance automatique du format et des colonnes
2. **Calculé les statistiques** : Moyenne, médiane, écart-type sans code manuel
3. **Agrégé par dimensions** : Totaux par région et par produit avec pandas
4. **Analysé la tendance** : Comparaison temporelle première/dernière semaine

**Avantages clés** :
- ✅ **Zéro code manuel** : Simple prompt en langage naturel
- ✅ **Formatage intelligent** : Résultats structurés et lisibles
- ✅ **Adaptabilité** : Fonctionne avec n'importe quel CSV similaire
- ✅ **Explications incluses** : L'assistant interprète les chiffres

**Note importante** : Le code généré par l'assistant est exécuté dans un environnement sandbox sécurisé. Vous ne voyez pas le code Python exécuté, seulement les résultats. Pour voir le code, vous pouvez demander explicitement "Montre-moi le code que tu as utilisé".

### Téléchargement et visualisation des images

Le Code Interpreter génère les graphiques comme fichiers PNG dans le sandbox. Ces images sont :
- **Accessibles via l'API** avec leur `file_id`
- **Téléchargeables** via `client.files.content()`
- **Temporaires** : supprimés après un certain délai

**Workflow de récupération** :
1. Parcourir les messages du thread
2. Identifier les contenus de type `image_file`
3. Extraire le `file_id`
4. Télécharger avec `files.content()`
5. Sauvegarder localement

**Astuce** : Vous pouvez demander à l'assistant de créer plusieurs visualisations en un seul prompt pour gagner du temps.

## Upload et analyse avec Code Interpreter

Pour analyser un fichier avec Code Interpreter :
1. **Upload** du fichier vers OpenAI avec `purpose='assistants'`
2. **Passage de l'ID** du fichier dans `tool_resources`
3. **Prompt** pour guider l'analyse

In [ ]:
# Uploader le fichier
with open('ventes_test.csv', 'rb') as f:
    file = client.files.create(file=f, purpose='assistants')

print(f"Fichier uploadé: {file.id}")
print(f"Taille: {file.bytes} bytes")

# Créer un assistant temporaire avec code_interpreter
# Note: L'Assistants API est en cours de dépréciation au profit de la Responses API.
# Cette approche fonctionne toujours mais des avertissements peuvent apparaître.
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

assistant = client.beta.assistants.create(
    name="Analyseur de ventes",
    instructions="Tu es un expert en analyse de données. Fournis des analyses détaillées et structurées.",
    model=DEFAULT_MODEL,
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}}
)

print(f"Assistant créé: {assistant.id}")

# Créer un thread et demander l'analyse
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""Analyse ce fichier CSV de ventes:
    1. Statistiques descriptives (moyenne, médiane, écart-type)
    2. Ventes totales par région
    3. Ventes totales par produit
    4. Tendance temporelle (première vs dernière semaine)
    Présente les résultats de façon structurée."""
)

# Exécuter l'analyse
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

print(f"\nStatut de l'exécution: {run.status}")

if run.status == 'completed':
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print("\n=== Analyse automatique ===")
    for msg in messages:
        if msg.role == "assistant":
            for content in msg.content:
                if hasattr(content, 'text'):
                    print(content.text.value)
else:
    print(f"Erreur: {run.status}")

### Interprétation de l'analyse automatique

Le Code Interpreter a effectué une analyse complète du fichier CSV sans code manuel. Voici ce qui s'est passé :

**Étapes automatiques réalisées** :
1. **Chargement** : Reconnaissance du format CSV et parsing automatique
2. **Exploration** : `df.describe()` pour statistiques descriptives
3. **Agrégation** : Groupement par région et par produit
4. **Analyse temporelle** : Comparaison première vs dernière semaine

**Résultats attendus** :
- Statistiques globales : moyenne (~321), médiane, min/max, écart-type
- Répartition par région : 4 totaux (un par région Nord/Sud/Est/Ouest)
- Répartition par produit : 3 totaux (A, B, C)
- Évolution temporelle : Tendance croissante/décroissante/stable

**Avantages du Code Interpreter** :
- ✅ **Zéro code manuel** : Simple prompt en langage naturel
- ✅ **Analyse intelligente** : Le modèle choisit les méthodes appropriées
- ✅ **Formatage structuré** : Résultats présentés clairement
- ✅ **Reproductibilité** : Même prompt → mêmes analyses

**Note technique** : Le statut `completed` confirme que l'exécution s'est terminée sans erreur dans le sandbox.

### Interprétation de la solution

Le système d'équations linéaires a été résolu avec **numpy.linalg.solve()**.

**Processus mathématique** :
1. Représentation matricielle : `Ax = b`
   - Matrice A (coefficients) : 3x3
   - Vecteur b (résultats) : 3x1
2. Résolution : `x = A⁻¹ × b` (inversion matricielle)
3. Validation : Vérifier que `Ax = b`

**Applications pratiques** :
- Économie : Équilibres de marché
- Physique : Lois de conservation
- Ingénierie : Circuits électriques, structures
- Optimisation : Points critiques

**Alternative** : Pour des systèmes plus complexes (non-linéaires), on utilise `scipy.optimize` ou des méthodes itératives.

### Interprétation des tests statistiques

Les résultats de l'analyse statistique permettent de répondre à plusieurs questions clés :

**1. Test de Shapiro-Wilk (normalité)** :
- **H0** : Les ventes suivent une distribution normale
- **Interprétation** : Si p > 0.05 → distribution normale, sinon non-normale
- **Impact** : Détermine quels tests statistiques sont appropriés

**2. ANOVA (comparaison régions)** :
- **H0** : Les moyennes de ventes sont identiques entre régions
- **Interprétation** : Si p < 0.05 → différences significatives entre régions
- **Usage** : Identifier les régions performantes/sous-performantes

**3. Test du Chi-2 (indépendance)** :
- **H0** : Région et produit sont indépendants
- **Interprétation** : Si p < 0.05 → certaines régions préfèrent certains produits
- **Usage** : Ciblage marketing, gestion des stocks

**4. Corrélation temporelle** :
- **Coefficient r** : Force et direction de la relation linéaire
- **Interprétation** : |r| proche de 1 → forte corrélation, proche de 0 → faible
- **Usage** : Détection de tendances (croissance/déclin)

**Note méthodologique** : Le seuil α = 0.05 (95% de confiance) est standard en sciences sociales. Pour des décisions critiques, utilisez α = 0.01 (99%).

## Génération de visualisations

Le Code Interpreter peut créer des graphiques avec matplotlib, seaborn ou plotly. Les images générées sont accessibles via l'API.

In [ ]:
# Créer un nouveau message pour demander des graphiques
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""Crée 2 graphiques à partir des données:
    1. Un graphique en barres des ventes totales par région
    2. Un graphique linéaire de la tendance des ventes dans le temps (avec moyenne mobile sur 7 jours)
    Utilise matplotlib et sauvegarde les images."""
)

# Exécuter
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

print(f"Statut: {run.status}")

if run.status == 'completed':
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print("\n=== Visualisations générées ===")
    
    for msg in messages:
        if msg.role == "assistant":
            for content in msg.content:
                # Texte
                if hasattr(content, 'text'):
                    print(content.text.value)
                
                # Images générées
                if hasattr(content, 'image_file'):
                    image_file_id = content.image_file.file_id
                    if image_file_id:  # Vérifier que le file_id n'est pas vide
                        print(f"\nImage générée: {image_file_id}")
                        
                        # Télécharger l'image
                        image_data = client.files.content(image_file_id)
                        image_path = f"graphique_{image_file_id[:8]}.png"
                        with open(image_path, 'wb') as f:
                            f.write(image_data.read())
                        print(f"Image sauvegardée: {image_path}")
                    else:
                        print("\n(Image référencée mais file_id non disponible)")
            break  # Prendre seulement le dernier message assistant

### Interprétation de la génération de visualisations

Le Code Interpreter a créé deux graphiques automatiquement :

**Graphique 1 : Ventes par région (barres)** :
- Visualisation comparative des performances régionales
- Permet d'identifier rapidement les régions sous-performantes ou sur-performantes
- Format idéal pour présentations (facile à lire)

**Graphique 2 : Tendance temporelle avec moyenne mobile** :
- Lissage de la courbe sur 7 jours pour éliminer le bruit
- Révèle la tendance de fond (croissance, stabilité, déclin)
- La moyenne mobile est calculée automatiquement par le Code Interpreter

**Processus de récupération des images** :
1. Parcourir les messages du thread
2. Identifier les contenus de type `image_file`
3. Extraire le `file_id`
4. Télécharger via `client.files.content(file_id)`
5. Sauvegarder localement en `.png`

**Note importante** : Les images sont **temporaires** dans le sandbox OpenAI. Il faut les télécharger immédiatement après génération, car elles peuvent être supprimées après un certain délai.

**Cas d'usage typiques** :
- Rapports automatisés (génération de graphiques standards)
- Dashboards dynamiques (visualisations basées sur données en temps réel)
- Exploration de données (essayer différents types de graphiques rapidement)

## Calculs mathématiques complexes

Le Code Interpreter peut résoudre des problèmes mathématiques avancés :
- Systèmes d'équations linéaires
- Calcul matriciel
- Intégration et dérivation symbolique
- Optimisation numérique

In [ ]:
# Créer un nouveau thread pour les calculs mathématiques
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

math_thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=math_thread.id,
    role="user",
    content="""Résous ce système d'équations linéaires avec numpy:
    2x + 3y - z = 1
    4x + y + 2z = 2
    -x + 2y + 3z = 3
    
    Montre le code utilisé et la solution complète."""
)

# Exécuter
run = client.beta.threads.runs.create_and_poll(
    thread_id=math_thread.id,
    assistant_id=assistant.id
)

print(f"Statut: {run.status}")

if run.status == 'completed':
    messages = client.beta.threads.messages.list(thread_id=math_thread.id)
    print("\n=== Résolution système d'équations ===")
    for msg in messages:
        if msg.role == "assistant":
            for content in msg.content:
                if hasattr(content, 'text'):
                    print(content.text.value)
            break

### Interprétation de la résolution du système d'équations

Le Code Interpreter a résolu le système d'équations linéaires automatiquement. Voici ce qui s'est passé :

**Système résolu** :
```
2x + 3y - z = 1
4x + y + 2z = 2
-x + 2y + 3z = 3
```

**Méthode utilisée** :
- Représentation matricielle : `Ax = b`
- Résolution avec `numpy.linalg.solve()` (inversion matricielle ou décomposition LU)
- Vérification : `A @ x ≈ b` (produit matriciel)

**Résultats attendus** :
- Trois valeurs numériques pour x, y, z
- Précision machine (~1e-15 pour la vérification)

**Pourquoi utiliser le Code Interpreter ?** :
- ✅ **Pas de setup manuel** : numpy déjà disponible dans le sandbox
- ✅ **Code généré automatiquement** : Le modèle écrit le code correct
- ✅ **Validation incluse** : Vérification automatique de la solution

**Applications pratiques** :
- Économie : Équilibres de marché (offre/demande multi-produits)
- Physique : Lois de Kirchhoff pour circuits électriques
- Chimie : Équilibres de réactions chimiques
- Ingénierie : Analyse de structures (forces, contraintes)

**Alternative pour systèmes complexes** : Pour des systèmes non-linéaires, utiliser `scipy.optimize.fsolve()` ou `sympy.solve()`.

## Exemple avancé : Analyse statistique

Testons une analyse plus approfondie avec tests d'hypothèses et corrélations.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""Effectue une analyse statistique avancée:
    1. Test de normalité des ventes (Shapiro-Wilk)
    2. ANOVA pour comparer les ventes entre régions
    3. Test du Chi-2 pour l'indépendance région/produit
    4. Corrélation entre le numéro du jour et les ventes
    
    Interprète chaque résultat avec un niveau de confiance à 95%."""
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

if run.status == 'completed':
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print("\n=== Analyse statistique avancée ===")
    for msg in messages:
        if msg.role == "assistant":
            for content in msg.content:
                if hasattr(content, 'text'):
                    print(content.text.value)
            break

### Interprétation de l'analyse statistique avancée

Le Code Interpreter a effectué une batterie complète de tests statistiques. Décryptons les résultats :

**1. Test de Shapiro-Wilk (normalité)** :
- **Hypothèse nulle (H0)** : Les ventes suivent une distribution normale
- **Seuil** : p > 0.05 → accepter H0 (distribution normale)
- **Interprétation** : Détermine si on peut utiliser des tests paramétriques (t-test, ANOVA)
- **Si p < 0.05** : Distribution non-normale → utiliser tests non-paramétriques (Mann-Whitney, Kruskal-Wallis)

**2. ANOVA (comparaison entre régions)** :
- **Hypothèse nulle (H0)** : Moyennes identiques entre toutes les régions
- **Seuil** : p < 0.05 → rejeter H0 (différences significatives)
- **Utilité** : Identifier si certaines régions performent mieux/moins bien
- **Post-hoc** : Si significatif, faire des tests par paires (Tukey HSD)

**3. Test du Chi-2 (indépendance région/produit)** :
- **Hypothèse nulle (H0)** : Région et produit sont indépendants
- **Seuil** : p < 0.05 → rejeter H0 (dépendance détectée)
- **Interprétation** : Certaines régions préfèrent certains produits
- **Application** : Ciblage marketing, gestion des stocks par région

**4. Corrélation temporelle (Pearson)** :
- **Coefficient r** : Entre -1 (anticorrélation) et +1 (corrélation parfaite)
- **Interprétation** :
  - |r| > 0.7 : Forte corrélation
  - 0.3 < |r| < 0.7 : Corrélation modérée
  - |r| < 0.3 : Faible corrélation
- **Usage** : Détecter tendances temporelles (croissance, saisonnalité)

**Niveau de confiance** :
- α = 0.05 (95% de confiance) est standard en sciences sociales
- Pour décisions critiques, utiliser α = 0.01 (99%) ou α = 0.001 (99.9%)

**Automatisation clé** : Le Code Interpreter choisit automatiquement les tests appropriés selon les données.

## Limitations et bonnes pratiques

### Limitations techniques

| Limitation | Impact | Solution |
|-----------|--------|----------|
| **Pas d'accès réseau** | Impossible de télécharger des données externes | Uploader tous les fichiers nécessaires |
| **Timeout ~120s** | Calculs longs peuvent échouer | Découper en étapes plus petites |
| **Bibliothèques limitées** | Certains packages spécialisés absents | Utiliser les alternatives standard |
| **Stockage temporaire** | Fichiers supprimés après session | Télécharger immédiatement les résultats |
| **Pas de GPU** | Pas d'accélération matérielle | Limiter la taille des modèles ML |

### Bonnes pratiques

1. **Vérifier les fichiers uploadés** : S'assurer que les données sont complètes
2. **Prompts précis** : Décrire exactement l'analyse souhaitée
3. **Télécharger rapidement** : Récupérer les images/fichiers générés avant expiration
4. **Validation externe** : Vérifier les résultats critiques manuellement
5. **Gérer les erreurs** : Prévoir des fallbacks si le code échoue

## Nettoyage des ressources

Important : supprimer les fichiers et assistants pour éviter les coûts de stockage.

In [ ]:
import os

# Supprimer l'assistant
try:
    client.beta.assistants.delete(assistant.id)
    print(f"Assistant {assistant.id} supprimé")
except Exception as e:
    print(f"Erreur suppression assistant: {e}")

# Supprimer le fichier uploadé
try:
    client.files.delete(file.id)
    print(f"Fichier {file.id} supprimé")
except Exception as e:
    print(f"Erreur suppression fichier: {e}")

# Supprimer le fichier local
if os.path.exists('ventes_test.csv'):
    os.remove('ventes_test.csv')
    print("Fichier local ventes_test.csv supprimé")

# Supprimer les images générées
for filename in os.listdir('.'):
    if filename.startswith('graphique_') and filename.endswith('.png'):
        os.remove(filename)
        print(f"Image {filename} supprimée")

print("\nNettoyage terminé !")

### Interprétation du nettoyage

Le nettoyage a été effectué avec succès. Voici ce qui a été supprimé :

**Ressources OpenAI** :
- ✅ **Assistant** : L'assistant temporaire a été supprimé (évite les coûts de stockage)
- ✅ **Fichier uploadé** : Le CSV a été retiré du serveur OpenAI

**Fichiers locaux** :
- ✅ **Dataset** : `ventes_test.csv` supprimé
- ✅ **Images générées** : Tous les fichiers `graphique_*.png` supprimés

**Pourquoi nettoyer ?** :
1. **Coûts** : Les fichiers stockés sur OpenAI peuvent entraîner des frais
2. **Espace disque** : Éviter l'accumulation de fichiers temporaires
3. **Sécurité** : Supprimer les données sensibles après utilisation
4. **Bonne pratique** : Toujours nettoyer après les tests/démonstrations

**Note importante** : Dans un environnement de production, vous voudriez peut-être :
- Sauvegarder les résultats importants avant de nettoyer
- Archiver les images générées dans un dossier dédié
- Logger les IDs des ressources supprimées pour audit

**Statut final** : Toutes les ressources temporaires ont été nettoyées. L'environnement est prêt pour une nouvelle session.

## Conclusion et exercices

### Ce que nous avons appris

Le **Code Interpreter** est un outil puissant pour :
- ✅ **Analyse de données** : Statistiques, exploration, nettoyage
- ✅ **Visualisations** : Graphiques automatiques avec matplotlib/seaborn
- ✅ **Calculs complexes** : Mathématiques, algèbre linéaire, optimisation
- ✅ **Automatisation** : Génération de rapports structurés

### Cas d'usage professionnels

1. **Business Intelligence** : Analyse de KPIs, reporting automatisé
2. **Data Science** : EDA rapide, feature engineering, preprocessing
3. **Finance** : Calculs actuariels, analyse de risque, backtesting
4. **Recherche** : Validation d'hypothèses, visualisation de résultats
5. **Éducation** : Démonstrations mathématiques, correction automatique

### Exercices suggérés

#### Exercice 1 : Analyse de timeseries
Créez un dataset de températures quotidiennes et demandez au Code Interpreter de :
- Détecter les anomalies (outliers)
- Calculer la moyenne mobile sur 7 jours
- Prédire les 7 prochains jours avec régression linéaire

#### Exercice 2 : Optimisation
Résolvez un problème d'optimisation linéaire avec scipy :
```
Maximiser: 3x + 2y
Contraintes:
  x + y <= 4
  2x + y <= 5
  x, y >= 0
```

#### Exercice 3 : Machine Learning basique
Uploadez un dataset de classification (iris, wine) et demandez :
- Entraînement d'un modèle k-NN
- Évaluation avec matrice de confusion
- Visualisation des frontières de décision

### Ressources complémentaires

- [OpenAI Code Interpreter Guide](https://platform.openai.com/docs/assistants/tools/code-interpreter)
- [Assistants API Reference](https://platform.openai.com/docs/api-reference/assistants)
- Notebook suivant : **8_Structured_Outputs.ipynb** (génération JSON validée)